<span style="color:Blue"><b> Student :</b> MILAN DHORE </span><br>
<span style="color:Blue"><b> Student ID:</b> 006104606 </span><br>
<span style="color:Blue"><b> Program :</b> Master of Science, Data Analytics </span><br>
<span style="color:Blue"><b> Program Mentor :</b> Jared Knepp </span><br>
<span style="color:Blue"><b> Course :</b> D208 -Task1 Multiple Linear Regression Performance Assessment(With Churn Dataset) </span><br>
<span style="color:Blue"><b> Course Instructor :</b> Eric Straw </span>

In [ ]:
# Standard data science imports
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# Visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Statistics packages
import pylab
from pylab import rcParams
import statsmodels.api as sm
import statistics
from scipy import stats

# Scikit-learn
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report


# Import chisquare from SciPy.stats
from scipy.stats import chisquare
from scipy.stats import chi2_contingency

# Ignore Warning Code
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [ ]:
# Load data set into Pandas dataframe and read first two rows including header
df = pd.read_csv('churn_raw_data.csv')
df.head(2)

In [ ]:
# Remove redundant "Unnamed" column at beginning & display first two records
df = df.drop(df.columns[0], axis = 1)
df.head(2)

In [ ]:
# Rename last 8 survey columns for better description of variables
df.rename(columns = {'item1':'Timely_Responses',
'item2':'Timely_Fixes',
'item3':'Timely_Replacements',
'item4':'Reliability',
'item5':'Options',
'item6':'Respectful_Response',
'item7':'Courteous_Exchange',
'item8':'Active_Listening'},
inplace=True)

<span style="color:Blue"><b> Data Cleaning </b></span>

In [ ]:
#Check remaining columns in new dataset and remove unwanted columns if required
df.columns

In [ ]:
#we should also check count of missing values as well "NA" presence in dataset.
data_with_nulls = df.isnull().sum() 
data_with_na=df.isna().any()
print(data_with_nulls)
print(data_with_na)

In [ ]:
# Identify duplicate rows in the dataframe and remove if required
duplicates_data_rows = df.loc[df.duplicated()]
print(duplicates_data_rows)

In [ ]:
# find out columns with invalid values or misspelling in case of catergorial data.
# This analysis will help later in analysis
print("Employment values \n",df['Employment'].unique())
print("Area values \n",df['Area'].unique())
print("Children values \n",df['Children'].unique())
print("Education values \n",df['Education'].unique())
print("Marital values \n",df['Marital'].unique())
print("Gender values \n",df['Gender'].unique())
print("Contract values \n",df['Contract'].unique())
print("PaymentMethod values \n",df['PaymentMethod'].unique())
print("age_range values \n",sorted(df['Age'].unique().astype(int)))

In [ ]:
# we found in above steps many missing data values for columns/features 
#Children, Age, Income, Tenure and Bandwidth_GB_Year
# Now its time to Impute missing fields  with median or mean
df['Children'] = df['Children'].fillna(df['Children'].median())
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Income'] = df['Income'].fillna(df['Income'].median())
df['Tenure'] = df['Tenure'].fillna(df['Tenure'].median())
df['Bandwidth_GB_Year'] = df['Bandwidth_GB_Year'].fillna(df['Bandwidth_GB_Year'].median())

In [ ]:
#RE-check count of missing values after imputing missing fields 
data_with_nulls = df.isnull().sum() 
print(data_with_nulls)

### <span style="color:Blue"><b> Dummy variable creation</b></span>

In [ ]:
##Turn all yes/no into dummy variables  
df['DummyGender'] = [1 if v == 'Male' else 0 for v in df['Gender']]
df['DummyChurn'] = [1 if v == 'Yes' else 0 for v in df['Churn']]
df['DummyTechie'] = [1 if v == 'Yes' else 0 for v in df['Techie']]
df['DummyContract'] = [1 if v == 'Two Year' else 0 for v in df['Contract']]
df['DummyPort_modem'] = [1 if v == 'Yes' else 0 for v in df['Port_modem']]
df['DummyTablet'] = [1 if v == 'Yes' else 0 for v in df['Tablet']]
df['DummyInternetService'] = [1 if v == 'Fiber Optic' else 0 for v in df['InternetService']]
df['DummyPhone'] = [1 if v == 'Yes' else 0 for v in df['Phone']]
df['DummyMultiple'] = [1 if v == 'Yes' else 0 for v in df['Multiple']]
df['DummyOnlineSecurity'] = [1 if v == 'Yes' else 0 for v in df['OnlineSecurity']]
df['DummyOnlineBackup'] = [1 if v == 'Yes' else 0 for v in df['OnlineBackup']]
df['DummyDeviceProtection'] = [1 if v == 'Yes' else 0 for v in df['DeviceProtection']]
df['DummyTechSupport'] = [1 if v == 'Yes' else 0 for v in df['TechSupport']]
df['DummyStreamingTV'] = [1 if v == 'Yes' else 0 for v in df['StreamingTV']]
df['StreamingMovies'] = [1 if v == 'Yes' else 0 for v in df['StreamingMovies']]
df['DummyPaperlessBilling'] = [1 if v == 'Yes' else 0 for v in df['PaperlessBilling']]

In [ ]:
# Drop original categorical features from dataframe
# Remove less meaningful demographic variables from statistics description

df = df.drop(columns=['Gender', 'Churn', 'Techie', 'Contract', 'Port_modem', 'Tablet', 
                    'InternetService', 'Phone', 'Multiple', 'OnlineSecurity', 
                    'OnlineBackup', 'DeviceProtection', 'TechSupport', 
                    'StreamingTV', 'StreamingMovies', 'PaperlessBilling',
                    'CaseOrder', 'Customer_id', 'Interaction', 'City', 'State', 'County',
       'Zip', 'Lat', 'Lng', 'Population', 'Area', 'Timezone', 'Job','Marital','PaymentMethod'])
df.describe()

In [ ]:
df.columns

In [ ]:
# Move Bandwidth_GB_Year to end of dataset as target
df_new = df[['Children', 'Age', 'Education', 'Employment', 'Income',
             'Outage_sec_perweek', 'Email', 'Contacts', 'Yearly_equip_failure',
             'Tenure', 'MonthlyCharge', 'Timely_Responses','Timely_Fixes', 
             'Timely_Replacements', 'Reliability', 'Options','Respectful_Response',
             'Courteous_Exchange', 'Active_Listening','DummyGender', 'DummyChurn',
             'DummyTechie', 'DummyContract','DummyPort_modem', 'DummyTablet',
             'DummyInternetService', 'DummyPhone','DummyMultiple', 
             'DummyOnlineSecurity', 'DummyOnlineBackup','DummyDeviceProtection', 
             'DummyTechSupport', 'DummyStreamingTV','DummyPaperlessBilling','Bandwidth_GB_Year']]

In [ ]:
df_new.head(2) 

### <span style="color:Blue"><b> Univariate Statistics </b></span>

In [ ]:
# Create histograms of contiuous variables
df_new[['Children', 'Age', 'Income', 'Outage_sec_perweek', 'Email', 
          'Contacts', 'Yearly_equip_failure', 'Tenure', 'MonthlyCharge', 
          'Bandwidth_GB_Year']].hist()
plt.savefig('churn_pyplot.jpg')
plt.tight_layout()

In [ ]:
# Create Seaborn boxplots for continuous variables
sns.boxplot('Tenure', data = df_new)
plt.show()

In [ ]:
sns.boxplot('MonthlyCharge', data = df_new)
plt.show()

In [ ]:
sns.boxplot('Bandwidth_GB_Year', data = df_new)
plt.show()

### <span style="color:Blue"><b> Bivariate Statistics </b></span>
Let's run some scatterplots to get an idea of our linear relationships with <br>our target variable of "Bandwidth_GB_Year" usage & some of the respective predictor variables.

In [ ]:
# Run scatterplots to show direct or inverse relationships 
# between target & independent variables
sns.scatterplot(x=df_new['Children'], y=df_new['Bandwidth_GB_Year'], color='red')
plt.show();

In [ ]:
sns.scatterplot(x=df_new['Age'], 
                y=df_new['Bandwidth_GB_Year'], color='red')
plt.show();

In [ ]:
sns.scatterplot(x=df_new['Income'], 
                y=df_new['Bandwidth_GB_Year'], color='red')
plt.show();

In [ ]:
sns.scatterplot(x=df_new['Outage_sec_perweek'], 
                y=df_new['Bandwidth_GB_Year'], color='red')
plt.show();

In [ ]:
sns.scatterplot(x=df_new['Email'], 
                y=df_new['Bandwidth_GB_Year'], color='red')
plt.show();

In [ ]:
sns.scatterplot(x=df_new['Contacts'], 
                y=df_new['Bandwidth_GB_Year'], color='red')
plt.show();

In [ ]:
sns.scatterplot(x=df_new['Yearly_equip_failure'], 
                y=df_new['Bandwidth_GB_Year'], color='red')
plt.show();

In [ ]:
sns.scatterplot(x=df_new['Tenure'], 
                y=df_new['Bandwidth_GB_Year'], color='red')
plt.show();

In [ ]:
sns.scatterplot(x=df_new['MonthlyCharge'],
                y=df_new['Bandwidth_GB_Year'], color='red')
plt.show();

In [ ]:
sns.scatterplot(x=df_new['Timely_Responses'], 
                y=df_new['Bandwidth_GB_Year'], color='red')
plt.show();

In [ ]:
sns.scatterplot(x=df_new['Timely_Fixes'], 
                y=df_new['Bandwidth_GB_Year'], color='red')
plt.show();

In [ ]:
sns.scatterplot(x=df_new['DummyTechie'], 
                y=df_new['Bandwidth_GB_Year'], color='red')
plt.show();

In [ ]:
## find correlation of cited some of the respective predictor variables
#with "Bandwidth_GB_Year"
plt.subplots(figsize=(15,15))
df_correlation = df[['Bandwidth_GB_Year','Children','Tenure','Age', 'Employment', 
                     'Income','Outage_sec_perweek', 'Email', 'Contacts', 'MonthlyCharge']]
sns.heatmap(df_correlation.corr(),annot=True,lw=1);

### <span style="color:Blue"><b> Prepared Dataset Copy  </b></span>

In [ ]:
# Extract Clean dataset
df_new.to_csv('churn_prepared_data.csv')

In [ ]:
df_prep=pd.read_csv('churn_prepared_data.csv')
# Remove redundant "Unnamed" column at beginning & display first two records
df_prep = df_prep.drop(df_prep.columns[0], axis = 1)
df_prep.head(2)

In [ ]:
df_prep.columns

In [ ]:
df_prep.shape

### <span style="color:Blue"><b> Initial Model  </b></span>

Significant advantage of the OLS model is it gives us the summary report of the model.It gives us a comprehensive report on how the model is split, what are the main parameters to look at, what are the different tests that is performed to validate if a feature is necessary or not for model creation.


In [ ]:
#Construct an initial multiple regression model from all predictors that were identified in Part C2.
#Develop the initial estimated regression equation that could be used to predict the Bandwidth_GB_Year
df['intercept'] = 1
bandwidth = sm.OLS(df['Bandwidth_GB_Year'], df[['Children', 'Age', 
                                                'Income', 'Outage_sec_perweek', 
                                                'Email', 'Contacts',
                                                'Yearly_equip_failure', 
                                                'DummyTechie', 'DummyContract', 
                                                'DummyPort_modem', 'DummyTablet', 
                                                'DummyInternetService', 'DummyPhone', 
                                                'DummyMultiple', 'DummyOnlineSecurity', 
                                                 'DummyOnlineBackup', 'DummyDeviceProtection', 
                                                 'DummyTechSupport', 'DummyStreamingTV', 
                                                 'DummyPaperlessBilling',
                                                'Tenure', 'MonthlyCharge', 
                                                 'Timely_Responses', 'Timely_Fixes', 
                                                 'Timely_Replacements', 'Reliability', 
                                                  'Options', 'Respectful_Response', 
                                                  'Courteous_Exchange', 'Active_Listening', 
                                                  'intercept']]).fit()
print(bandwidth.summary())

Based on an R2 value = 0.812. So, 81% of the variation is explained by this model. The condition number is large which might suggest strong multicollinearity. Apparently, we do not need all these variables to explain the variance. So, it’s a very well fitted model where we don’t have any such feature which is affecting our model performance out of proportion

###  <span style="color:Blue"><b> Justification of Model Reduction</b></span>
Justify a statistically based variable selection procedure and a model evaluation metric to reduce the initial model in a way that aligns with the research question.

In [ ]:
# Create dataframe for heatmap bivariate analysis of correlation3
# Only considered variables with positive coefficient 
df_bivariate=df[['Bandwidth_GB_Year','Children', 'Tenure',
                 'DummyOnlineSecurity','DummyDeviceProtection'
                 ,'DummyDeviceProtection','DummyStreamingTV',
                 'MonthlyCharge','Courteous_Exchange']]

In [ ]:
# Generate heatmap
sns.heatmap(df_bivariate.corr(), annot=True)
plt.show()

<b>Again, it appears that Tenure is the predictor for most of the variance. There is clearly a direct linear relationship between customer tenure with the telecom company & the amount of data (in GBs) that is being used.</b>


###  <span style="color:Blue"><b> Reduced Multiple Regression Model</b></span>

We have selected two continuous variables Children & Tenure as well as two discrete ordinal predictor variables Timely Fixes and courteous exchange

In [52]:
# Run reduced OLS multiple regression
df['intercept'] = 1
lm_bandwidth_reduced = sm.OLS(df['Bandwidth_GB_Year'], df[['Children',
                                                           'Tenure', 
                                                           'Timely_Fixes', 
                                                           'Courteous_Exchange',
                                                           'intercept']]).fit()
print(lm_bandwidth_reduced.summary())

                            OLS Regression Results                            
Dep. Variable:      Bandwidth_GB_Year   R-squared:                       0.800
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                     9990.
Date:                Fri, 22 Oct 2021   Prob (F-statistic):               0.00
Time:                        17:36:47   Log-Likelihood:                -82510.
No. Observations:               10000   AIC:                         1.650e+05
Df Residuals:                    9995   BIC:                         1.651e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Children              23.9622      4

<b>After Removing all those other predictor variables also our model still explains 80% of the variance
</b>

###  <span style="color:Blue"><b> Residual Plot</b></span>
The residual plots can reveal conditions that are hard to see from the regression line

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
#fit multiple linear regression model
residuals = ols('Bandwidth_GB_Year ~ Children + Tenure + Timely_Fixes + Courteous_Exchange ', data=df).fit()

#view model summary
print(residuals.summary())

In [ ]:
#create residual vs. predictor plot for 'Children'
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(residuals, 'Children', fig=fig)

In [ ]:
#create residual vs. predictor plot for 'Tenure'
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(residuals, 'Tenure', fig=fig)


In [ ]:
#create residual vs. predictor plot for 'Timely_Fixes'
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(residuals, 'Timely_Fixes', fig=fig)

In [ ]:
#create residual vs. predictor plot for 'Courteous_Exchange'
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(residuals, 'Courteous_Exchange', fig=fig)